In [3]:
import pandas as pd

In [2]:
data_path=r"C:\Users\gauta\OneDrive\Desktop\Untitled Folder\MapUp-Data-Assessment-F-main\datasets\dataset-1.csv"

In [3]:
data=pd.read_csv(data_path)

In [4]:
data.head()

,id_1,id_2,route,moto,car,rv,bus,truck
0,829,827,1,2.05,4.14,4.14,10.1,15.2
1,829,821,4,6.63,13.26,13.26,32.4,48.5
2,829,804,7,14.41,28.92,28.92,64.7,97.0
3,829,822,6,5.90,11.81,11.81,28.8,43.2
4,829,826,9,2.87,5.81,5.81,14.2,21.2


In [8]:
def generate_car_matrix(df):
    # creating pivot the DataFrame 
    df = df.pivot(index='id_1', columns='id_2', values='car').fillna(0)
    
    # Replace the diagonal values with 0
    for i in range(min(df.shape)):
        df.iloc[i, i] = 0
    
    return df


In [9]:
result_matrix = generate_car_matrix(data)


In [10]:
result_matrix.head()

id_2,801,802,803,804,805,806,807,808,809,821,822,823,824,825,826,827,829,830,831
id_1,,,,,,,,,,,,,,,,,,,
801,0.0,2.8,6.0,7.7,11.7,13.4,16.9,19.6,21.0,23.52,24.67,26.53,27.92,29.08,30.87,32.53,36.32,38.27,39.24
802,2.8,0.0,3.4,5.2,9.2,10.9,14.3,17.1,18.5,20.92,22.07,23.93,25.32,26.48,28.27,29.93,33.72,35.67,36.64
803,6.0,3.4,0.0,2.0,6.0,7.7,11.1,13.9,15.3,17.72,18.87,20.73,22.12,23.28,25.07,26.73,30.52,32.47,33.44
804,7.7,5.2,2.0,0.0,4.4,6.1,9.5,12.3,13.7,16.12,17.27,19.13,20.52,21.68,23.47,25.13,28.92,30.87,31.84
805,11.7,9.2,6.0,4.4,0.0,2.0,5.4,8.2,9.6,12.02,13.17,15.03,16.42,17.58,19.37,21.03,24.82,26.77,27.74


In [17]:
def get_type_count(df)->dict:
    #adding new col to the dataset
    df['car_type'] = pd.cut(df['car'], bins=[-float('inf'), 15, 25, float('inf')],
                            labels=['low', 'medium', 'high'], right=False)

    #calculate the count of occurrences for each 'car_type' category
    type_counts = df['car_type'].value_counts().to_dict()

    #sort the dictionary alphabetically based on keys
    ans = sorted(type_counts.items())

    return dict(ans)


In [18]:
result_counts = get_type_count(data)
print(result_counts)

{'high': 56, 'low': 196, 'medium': 89}


In [19]:
def get_bus_indexes(df)->list:
    #calculate the mean
    bus_mean = df['bus'].mean()
    
    #identify indices where 'bus' values are greater than twice 
    bus_indexes = df[df['bus'] > 2 * bus_mean].index.tolist()
    
    #sort the indices in ascending order
    bus_indexes.sort()
    
    return list(bus_indexes)


In [20]:
result_indexes = get_bus_indexes(data)
print(result_indexes)

[2, 7, 12, 17, 25, 30, 54, 64, 70, 97, 144, 145, 149, 154, 160, 201, 206, 210, 215, 234, 235, 245, 250, 309, 314, 319, 322, 323, 334, 340]


In [21]:
def filter_routes(df)->list:
    #Group the DataFrame by 'route' and calculate the average of 'truck' column for each route
    route_avg_truck = df.groupby('route')['truck'].mean()
    
    #Filter routes where the average of 'truck' column is greater than 7
    filtered_routes = route_avg_truck[route_avg_truck > 7].index.tolist()
    
    #Sort the list of filtered routes
    filtered_routes.sort()
    
    return list(filtered_routes)


In [22]:
filtered_routes_list=filter_routes(data)
print(filtered_routes_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [23]:
def multiply_matrix(df)->pd.DataFrame:
    #Deep copy the input DataFrame to avoid modifying the original DataFrame
    df = df.copy(deep=True)
    
    #Apply the specified logic to modify values in the DataFrame
    df = df.applymap(lambda x: x * 0.75 if x > 20 else x * 1.25)
    
    #Round the modified values to 1 decimal place
    df = df.round(1)
    
    return df


In [24]:
modified_result_df = multiply_matrix(result_matrix)
print(modified_result_df)

id_2   801   802   803   804   805   806   807   808   809   821   822   823  \
id_1                                                                           
801    0.0   3.5   7.5   9.6  14.6  16.8  21.1  24.5  15.8  17.6  18.5  19.9   
802    3.5   0.0   4.2   6.5  11.5  13.6  17.9  21.4  23.1  15.7  16.6  17.9   
803    7.5   4.2   0.0   2.5   7.5   9.6  13.9  17.4  19.1  22.2  23.6  15.5   
804    9.6   6.5   2.5   0.0   5.5   7.6  11.9  15.4  17.1  20.2  21.6  23.9   
805   14.6  11.5   7.5   5.5   0.0   2.5   6.8  10.2  12.0  15.0  16.5  18.8   
806   16.8  13.6   9.6   7.6   2.5   0.0   4.8   8.2  10.0  13.0  14.5  16.8   
807   21.1  17.9  13.9  11.9   6.8   4.8   0.0   3.6   5.4   8.5  10.0  12.3   
808   24.5  21.4  17.4  15.4  10.2   8.2   3.6   0.0   2.1   5.2   6.6   8.9   
809   15.8  23.1  19.1  17.1  12.0  10.0   5.4   2.1   0.0   3.6   5.1   7.4   
821   17.6  15.7  22.2  20.2  15.0  13.0   8.5   5.2   3.6   0.0   2.2   4.6   
822   18.5  16.6  23.6  21.6  16.5  14.5

In [26]:
data2_path=r"C:\Users\gauta\OneDrive\Desktop\Untitled Folder\MapUp-Data-Assessment-F-main\datasets\dataset-2.csv"

In [27]:
data2=pd.read_csv(data2_path)

In [28]:
data2.head()

,id,name,id_2,startDay,startTime,endDay,endTime,able2Hov2,able2Hov3,able3Hov2,able3Hov3,able5Hov2,able5Hov3,able4Hov2,able4Hov3
0,1040000,Montgomery,-1,Monday,05:00:00,Wednesday,10:00:00,3.0,3.0,-1.0,-1,3,3,3,3
1,1040010,Black,-1,Monday,10:00:00,Friday,15:00:00,6.0,6.0,-1.0,-1,6,6,6,6
2,1040020,Emerald,-1,Thursday,15:00:00,Friday,19:00:00,3.0,3.0,-1.0,-1,3,3,3,3
3,1040030,Foley,-1,Monday,19:00:00,Friday,23:59:59,6.0,6.0,-1.0,-1,6,6,6,6
4,1050000,Whittier,1050001,Saturday,00:00:00,Sunday,23:59:59,6.0,6.0,NaN,-1,6,6,6,6


In [37]:
def verify_time_completeness(df):
    try:
        # Combine date and time columns to create datetime columns
        df['start_datetime'] = pd.to_datetime(df['startDay'] + ' ' + df['startTime'], errors='coerce')
        df['end_datetime'] = pd.to_datetime(df['endDay'] + ' ' + df['endTime'], errors='coerce')

        # Drop rows with invalid timestamps
        df = df.dropna(subset=['start_datetime', 'end_datetime'])

        # Calculate time differences
        df['time_diff'] = (df['end_datetime'] - df['start_datetime']).dt.total_seconds()

        # Group by (id, id_2) pairs and check completeness of timestamps
        completeness_check = df.groupby(['id', 'id_2']).apply(
            lambda x: (
                (x['time_diff'].min() >= 0) and 
                (x['time_diff'].max() >= 86399) and 
                (x['start_datetime'].dt.weekday.min() == 0) and 
                (x['start_datetime'].dt.weekday.max() == 6)
            )  
        )

        return completeness_check
    except Exception as e:
        print("Error:", e)
        return pd.Series()


In [38]:
completeness_series = verify_time_completeness(data2)
print(completeness_series)


Empty DataFrame
Columns: [id, name, id_2, startDay, startTime, endDay, endTime, able2Hov2, able2Hov3, able3Hov2, able3Hov3, able5Hov2, able5Hov3, able4Hov2, able4Hov3, start_datetime, end_datetime, time_diff]
Index: []
